In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split #split data into train and test sets
from sklearn.feature_extraction.text import CountVectorizer #convert text comment into a numeric vector
from sklearn.feature_extraction.text import TfidfTransformer #use TF IDF transformer to change text vector created by count vectorizer
from sklearn.svm import SVC# Support Vector Machine
from sklearn.pipeline import Pipeline #pipeline to implement steps in series
from gensim import parsing # To stem data

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


#Read csv into a dataframe
df = pd.read_csv("./train.csv")
#print first 5 rows of dataset
print(df.head())

# Any results you write to the current directory are saved as output.

   Unnamed: 0                                              title  \
0           0  Obama’s Delusion Continues In Vapid Address To...   
1           1  Huge blast in China's Ningbo city kills at lea...   
2           2  Trump applauds senators for new healthcare ref...   
3           3  UAW chief says Clinton told him she would rene...   
4           4  The Las Vegas Mass Shooting – More to the Stor...   

                                                text       subject  \
0  Obama addressed the Nation In a nothing burger...      politics   
1  BEIJING (Reuters) - A powerful explosion in a ...     worldnews   
2  WASHINGTON (Reuters) - President Donald Trump ...  politicsNews   
3  (Reuters) - United Auto Workers President Denn...  politicsNews   
4  Shawn Helton 21st Century WireAlthough many ar...       US_News   

                  date  label  
0          Dec 6, 2015      0  
1   November 26, 2017       1  
2  September 13, 2017       1  
3       July 26, 2016       1  
4     Octo

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:

from sklearn.metrics import classification_report, confusion_matrix

#for grouping similar words such as 'trying" and "try" are same words
def parse(s):
    parsing.stem_text(s)
    return s

#applying parsing to comments.
for i in range(0,len(df)):
    df.iloc[i,2]=parse(df.iloc[i,2])
    
#Seperate data into feature and results
X, y = df['text'].tolist(), df['label'].tolist()

#Split data in train and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)



#Use pipeline to carry out steps in sequence with a single object
#SVM's rbf kernel gives highest accuracy in this classification problem.
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SVC(kernel='rbf'))])

#train model
text_clf.fit(X_train, y_train)

ytest = np.array(y_test)

# #predict class form test data 
# predicted = text_clf.predict(X_test)


# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, text_clf.predict(X_test)))
print(confusion_matrix(ytest, text_clf.predict(X_test)))



              precision    recall  f1-score   support

           0       1.00      0.73      0.84        11
           1       0.82      1.00      0.90        14

    accuracy                           0.88        25
   macro avg       0.91      0.86      0.87        25
weighted avg       0.90      0.88      0.88        25

[[ 8  3]
 [ 0 14]]
